
# Table of Contents



Initialize Python session with imports.



In [1]:
using BenchmarkTools
using DelimitedFiles
using Test
@btime 0;

0.018 ns (0 allocations: 0 bytes)

## 11 Largest product in a grid



In the 20×20 grid below, four numbers along a diagonal line have been marked in
<del>red</del>.



In [1]:
A = """
 8  2 22 97 38 15  0 40  0 75  4  5  7 78 52 12 50 77 91  8
49 49 99 40 17 81 18 57 60 87 17 40 98 43 69 48  4 56 62  0
81 49 31 73 55 79 14 29 93 71 40 67 53 88 30  3 49 13 36 65
52 70 95 23  4 60 11 42 69 24 68 56  1 32 56 71 37  2 36 91
22 31 16 71 51 67 63 89 41 92 36 54 22 40 40 28 66 33 13 80
24 47 32 60 99  3 45  2 44 75 33 53 78 36 84 20 35 17 12 50
32 98 81 28 64 23 67 10 26 38 40 67 59 54 70 66 18 38 64 70
67 26 20 68  2 62 12 20 95 63 94 39 63  8 40 91 66 49 94 21
24 55 58  5 66 73 99 26 97 17 78 78 96 83 14 88 34 89 63 72
21 36 23  9 75  0 76 44 20 45 35 14  0 61 33 97 34 31 33 95
78 17 53 28 22 75 31 67 15 94  3 80  4 62 16 14  9 53 56 92
16 39  5 42 96 35 31 47 55 58 88 24  0 17 54 24 36 29 85 57
86 56  0 48 35 71 89  7  5 44 44 37 44 60 21 58 51 54 17 58
19 80 81 68  5 94 47 69 28 73 92 13 86 52 17 77  4 89 55 40
 4 52  8 83 97 35 99 16  7 97 57 32 16 26 26 79 33 27 98 66
88 36 68 87 57 62 20 72  3 46 33 67 46 55 12 32 63 93 53 69
 4 42 16 73 38 25 39 11 24 94 72 18  8 46 29 32 40 62 76 36
20 69 36 41 72 30 23 88 34 62 99 69 82 67 59 85 74  4 36 16
20 73 35 29 78 31 90  1 74 31 49 71 48 86 81 16 23 57  5 54
1 70 54 71 83 51 54 69 16 92 33 48 61 43 52  1 89 19 67 48
"""

A = readdlm(IOBuffer(A), Int);

The product of these numbers is 26 × 63 × 78 × 14 = 1788696.

What is the greatest product of four adjacent numbers in the same direction (up,
down, left, right, or diagonally) in the 20×20 grid?



### A:



Consider $A\in\mathbb{Z}^{n\times m}$:

$$ A = \begin{bmatrix}{}
       a_{1,1} & a_{1,2} & \dots  & a_{1,m} \\
       a_{2,1} & a_{2,2} & \dots  & a_{2,m} \\
       \vdots  & \vdots  & \ddots & \vdots \\
       a_{n,1} & a_{n,2} & \dots  & a_{n,m}
       \end{bmatrix} $$

For a given window size $w$, define matrices for each of the four unique
directions of adjacency:

1.  $\mathbf{V}^w \in\mathbb{Z}^{(n-w+1) \times  m   }$ for the vertical products,
2.  $\mathbf{H}^w \in\mathbb{Z}^{ n      \times (m-w+1)}$ for the horizontal products,
3.  $\mathbf{B}^w \in\mathbb{Z}^{(n-w+1) \times (m-w+1)}$ for the \`\\' (backslash-like) products,
4.  $\mathbf{F}^w \in\mathbb{Z}^{(n-w+1) \times (m-w+1)}$ for the \`/' (frontslash-like) products.

where the elements of these matrices are

\begin{eqnarray*}
v^w_{i,j} & = & \prod_{k=0}^{w-1} a_{i+k, j    } \\
h^w_{i,j} & = & \prod_{k=0}^{w-1} a_{i,   j+k  } \\
b^w_{i,j} & = & \prod_{k=0}^{w-1} a_{i+k, j+k  } \\
f^w_{i,j} & = & \prod_{k=0}^{w-1} a_{i+k, j+w-k}
\end{eqnarray*}

And construct $\mathbf{V,H,B,F}$:



In [1]:
"Find all the adjacent products in matrix ``A`` of window size ``w``."
function adjacentproducts(A::Array{T,2}, w::Integer) where {T<:Integer}
    n, m = size(A)
    V = ones(T,n-w+1,m)
    H = ones(T,n,m-w+1)
    B = ones(T,n-w+1,m-w+1)
    F = ones(T,n-w+1,m-w+1)
    for k in 1:w
        V .*= A[k:end+k-w,:]
        H .*= A[:,k:end+k-w]
        B .*= A[k:end+k-w,k:end+k-w]
        F .*= A[k:end+k-w,w-k+1:end-k+1]
    end
    return V, H, B, F
end

w = 4
V, H, B, F = adjacentproducts(A,w)
@test B[7,9] == 1788696

@btime maximum([maximum(M) for M in adjacentproducts(A,w)])

15.496 μs (24 allocations: 101.48 KiB)
70600674

## 12 Highly divisible triangular number



The sequence of triangle numbers is generated by adding the natural
numbers. So the 7th triangle number would be 1 + 2 + 3 + 4 + 5 + 6 + 7 = 28. The
first ten terms would be:

1, 3, 6, 10, 15, 21, 28, 36, 45, 55, &#x2026;

Let us list the factors of the first seven triangle numbers:

| $T_i$|$factors(T_i)$|
|---|---|
| 1|1|
| 3|1,3|
| 6|1,2,3,6|
| 10|1,2,5,10|
| 15|1,3,5,15|
| 21|1,3,7,21|
| 28|1,2,4,7,14,28|

We can see that 28 is the first triangle number to have over five divisors.

What is the value of the first triangle number to have over five hundred
divisors?



### A:



We first need some way to sieve for the number of divisors.



In [1]:
"Find the number of divisors for all nonnegative numbers less than or equal to ``n``."
function ndivisors(n::T) where {T<:Integer}
    d = zeros(T,n)
    nhalf = trunc(T,n/2)
    for i in 1:nhalf
        d[i:i:end] .+= 1
    end
    d[nhalf+1:end] .+= 1
    return d
end

@test ndivisors(9) == [1,2,2,3,2,4,2,4,3]

Test Passed

We know that $T_n = \frac{n(n+1)}{2}$ (see [problem 1](001.md)) and that all $n,n+1$
are coprime. Therefore, we can break the problem into smaller problems. First,
write $T_n$ as the product of two coprime integers, keeping in mind that $n$
and any factors of $n+1$ are coprime, and $n+1$ and any factors of $n$ are
coprime.

$$ T_n =
    \begin{cases}
        n\cdot\frac{n+1}{2}, & \text{n is odd} \\
        \frac{n}{2}\cdot(n+1), & \text{n is even}
    \end{cases} $$

Consider two coprimes, $n,n'$. They have prime factorizations

$$ n = \prod_i p_i^{a_i} \\
   n' = \prod_j p_j^{a_j} $$

such that $p_i\neq p_j \,\forall\, i,j$. Therefore, the prime factorization
of $n\cdot n'$ is

$$ n\cdot n' = \left[ \prod_i p_i^{a_i} \right] \cdot
               \left[ \prod_j p_j^{a_j} \right] $$

Now, the number of divisors can be found from the prime factorization easily
using combinatorics. If we construct a divisor, $d|n$, it will have a prime
factorization

$$ d = \prod_i p_i^{b_i} $$

such that $0\leq b_i\leq a_i \,\forall\, i$. In other words, for each exponent
of $p_i$ we can choose from $0,1,...,a_i$ for the exponent in the divisor.
There are $a_i+1$ choices for every prime factor of $n$, making the total
number of unique divisors we can construct

$$ D(n) = \prod_i a_i+1 $$

Additionally, since coprimes cannot share prime factors, the number of divisors
for the product of coprimes is a multiplicative function.

$$ D(n\cdot n') = \left[ \prod_i a_i+1 \right] \cdot
                  \left[ \prod_j a_j+1 \right] = D(n)D(n') $$

We already know $T_n$ is a product of coprimes. Therefore,

$$ D(T_n) =
    \begin{cases}
        D(n)D((n+1)/2), & \text{n is odd} \\
        D(n/2)D(n+1), & \text{n is even}
    \end{cases} $$

Now we have a way to iterate over $i$ and find $D(T_i)$ without explicitly
calculating the number of divisors of $T_i$.

I'll guess 15000 as the highest index our number could possibly be (it is
actually a bit less). Our algorithm takes $O(n)$ time, where $n$ is the
index of the triangular number we guess. If we had sieved over all $T_n$, this
would have taken $O(T_n)=O(n^2)$ time.



In [1]:
"Find the ``n``th triangular number."
triangular(n::T) where {T<:Integer} = convert(T,div(n*(n+1),2))

"""
Find the first triangular number with more than ``k`` divisors. Requires a limit
with which to generate the sieve of number of divisors.
"""
function pe012(k::Integer,ndiv::Vector{T}) where {T<:Integer}
    for i in 1:(length(ndiv)-2)
        dtri = (i%2 == 0 ? ndiv[trunc(T,i/2)]*ndiv[i+1] : ndiv[i]*ndiv[trunc(T,(i+1)/2)])
        if dtri > k
            return triangular(T(i))
        end
    end
    return nothing
end
pe012(k::Integer,limit::Integer) = pe012(k,ndivisors(limit))

@test pe012(5,10) == 28

@btime pe012(500,15000)

529.337 μs (7511 allocations: 1.84 MiB)
76576500

## 13 Large sum



Work out the first ten digits of the sum of the following one-hundred
50-digit numbers.



In [1]:
numbers = [
  37107287533902102798797998220837590246510135740250,
  46376937677490009712648124896970078050417018260538,
  74324986199524741059474233309513058123726617309629,
  91942213363574161572522430563301811072406154908250,
  23067588207539346171171980310421047513778063246676,
  89261670696623633820136378418383684178734361726757,
  28112879812849979408065481931592621691275889832738,
  44274228917432520321923589422876796487670272189318,
  47451445736001306439091167216856844588711603153276,
  70386486105843025439939619828917593665686757934951,
  62176457141856560629502157223196586755079324193331,
  64906352462741904929101432445813822663347944758178,
  92575867718337217661963751590579239728245598838407,
  58203565325359399008402633568948830189458628227828,
  80181199384826282014278194139940567587151170094390,
  35398664372827112653829987240784473053190104293586,
  86515506006295864861532075273371959191420517255829,
  71693888707715466499115593487603532921714970056938,
  54370070576826684624621495650076471787294438377604,
  53282654108756828443191190634694037855217779295145,
  36123272525000296071075082563815656710885258350721,
  45876576172410976447339110607218265236877223636045,
  17423706905851860660448207621209813287860733969412,
  81142660418086830619328460811191061556940512689692,
  51934325451728388641918047049293215058642563049483,
  62467221648435076201727918039944693004732956340691,
  15732444386908125794514089057706229429197107928209,
  55037687525678773091862540744969844508330393682126,
  18336384825330154686196124348767681297534375946515,
  80386287592878490201521685554828717201219257766954,
  78182833757993103614740356856449095527097864797581,
  16726320100436897842553539920931837441497806860984,
  48403098129077791799088218795327364475675590848030,
  87086987551392711854517078544161852424320693150332,
  59959406895756536782107074926966537676326235447210,
  69793950679652694742597709739166693763042633987085,
  41052684708299085211399427365734116182760315001271,
  65378607361501080857009149939512557028198746004375,
  35829035317434717326932123578154982629742552737307,
  94953759765105305946966067683156574377167401875275,
  88902802571733229619176668713819931811048770190271,
  25267680276078003013678680992525463401061632866526,
  36270218540497705585629946580636237993140746255962,
  24074486908231174977792365466257246923322810917141,
  91430288197103288597806669760892938638285025333403,
  34413065578016127815921815005561868836468420090470,
  23053081172816430487623791969842487255036638784583,
  11487696932154902810424020138335124462181441773470,
  63783299490636259666498587618221225225512486764533,
  67720186971698544312419572409913959008952310058822,
  95548255300263520781532296796249481641953868218774,
  76085327132285723110424803456124867697064507995236,
  37774242535411291684276865538926205024910326572967,
  23701913275725675285653248258265463092207058596522,
  29798860272258331913126375147341994889534765745501,
  18495701454879288984856827726077713721403798879715,
  38298203783031473527721580348144513491373226651381,
  34829543829199918180278916522431027392251122869539,
  40957953066405232632538044100059654939159879593635,
  29746152185502371307642255121183693803580388584903,
  41698116222072977186158236678424689157993532961922,
  62467957194401269043877107275048102390895523597457,
  23189706772547915061505504953922979530901129967519,
  86188088225875314529584099251203829009407770775672,
  11306739708304724483816533873502340845647058077308,
  82959174767140363198008187129011875491310547126581,
  97623331044818386269515456334926366572897563400500,
  42846280183517070527831839425882145521227251250327,
  55121603546981200581762165212827652751691296897789,
  32238195734329339946437501907836945765883352399886,
  75506164965184775180738168837861091527357929701337,
  62177842752192623401942399639168044983993173312731,
  32924185707147349566916674687634660915035914677504,
  99518671430235219628894890102423325116913619626622,
  73267460800591547471830798392868535206946944540724,
  76841822524674417161514036427982273348055556214818,
  97142617910342598647204516893989422179826088076852,
  87783646182799346313767754307809363333018982642090,
  10848802521674670883215120185883543223812876952786,
  71329612474782464538636993009049310363619763878039,
  62184073572399794223406235393808339651327408011116,
  66627891981488087797941876876144230030984490851411,
  60661826293682836764744779239180335110989069790714,
  85786944089552990653640447425576083659976645795096,
  66024396409905389607120198219976047599490197230297,
  64913982680032973156037120041377903785566085089252,
  16730939319872750275468906903707539413042652315011,
  94809377245048795150954100921645863754710598436791,
  78639167021187492431995700641917969777599028300699,
  15368713711936614952811305876380278410754449733078,
  40789923115535562561142322423255033685442488917353,
  44889911501440648020369068063960672322193204149535,
  41503128880339536053299340368006977710650566631954,
  81234880673210146739058568557934581403627822703280,
  82616570773948327592232845941706525094512325230608,
  22918802058777319719839450180888072429661980811197,
  77158542502016545090413245809786882778948721859617,
  72107838435069186155435662884062257473692284509516,
  20849603980134001723930671666823555245252804609722,
  53503534226472524250874054075591789781264330331690
];

### A:



Julia has support for large integers, making this problem trivial.



In [1]:
@btime string(sum(numbers))[1:10]

1.186 μs (7 allocations: 256 bytes)
"5537376230"

## 14 Longest Collatz sequence



The following iterative sequence is defined for the set of positive integers:

n → n/2 (n is even)
n → 3n + 1 (n is odd)

Using the rule above and starting with 13, we generate the following sequence:

13 → 40 → 20 → 10 → 5 → 16 → 8 → 4 → 2 → 1
It can be seen that this sequence (starting at 13 and finishing at 1) contains
10 terms. Although it has not been proved yet (Collatz Problem), it is thought
that all starting numbers finish at 1.

Which starting number, under one million, produces the longest chain?

NOTE: Once the chain starts the terms are allowed to go above one million.



### A:



The [hackerrank version](https://www.hackerrank.com/contests/projecteuler/challenges/euler014/problem) ups the limit to $5\times 10^6$ and requires up to
$10^4$ trials, all completing in just a few seconds. Furthermore, there's the
issue that multiple numbers can have the same sequence length, and hackerrank
asks for the greatest of those, not the least. In other words, we want the
version of [A006877](https://oeis.org/A006877) that includes sequences that tie with the last record.

First, we need a helper function to get the next number in the sequence.
Colloquially, the Collatz sequence is known as the hailstone sequence.



In [1]:
"Finds the next number in the Collatz sequence using the hailstone algorithm."
hailstone(n::T) where {T<:Integer} = (n%2 == 1 ? T(3n+1) : n >> 1)

@test hailstone(2) == 1
@test hailstone(3) == 10
@test hailstone(4) == 2
@test hailstone(5) == 16

Test Passed

Second, we need a way to find the length of a Collatz sequence and a way to
cache the results.



In [1]:
"""
Helper function for ``A006877``. Finds the length of a Collatz sequence begging
with ``n``. If a ``cache=[1 0 0...]`` is supplied, the depths are cached as they
are calculated recursively.
"""
collatzdepth(n::T) where {T<:Integer} = (n==1 ? T(1) : collatzdepth(hailstone(n)) + T(1))
function collatzdepth(n::Integer, cache::Array{T}) where {T<:Integer}
    if n > length(cache)
        return collatzdepth(hailstone(n),cache) + 1
    end
    if cache[n] == 0
        cache[n] = collatzdepth(hailstone(n),cache) + 1
    end
    return cache[n]
end

cache = zeros(Int,10)
cache[1] = 1
@test collatzdepth(10) == 7
@test collatzdepth(10,cache) == 7
@test cache == [1,2,0,3,6,0,0,4,0,7]

Test Passed

Last, the function `A006877` computes the sequence up to a certain limit
(ties included). A cache, `cache::Array{Int}`, is passed to the `collatzdepth`
function. This function computes the depth for a given starting sequence,
recursively, caches the intermediate lengths it finds while computing the
sequence, and terminates if it can find the depth of the remaining sequence in
the cache. This is essentially the same method as I used for Python, but the
program is ten times faster.



In [1]:
"""
Find the starting numbers, not exceeding ``limit``, that produce record-breaking
or record-tieing Collatz sequence lengths.
"""
function A006877(limit::T) where {T<:Integer}
    cache = zeros(T,limit)
    cache[1] = 1
    dmax = T[1]
    last = collatzdepth(dmax[end],cache)
    for i in 2:limit
        if collatzdepth(i, cache) >= last
            push!(dmax,i)
            last = collatzdepth(dmax[end],cache)
        end
    end
    return dmax
end

@test A006877(10) == [1,2,3,6,7,9]

L = 10^6
@btime A006877(L)[end]

20.485 ms (9 allocations: 7.63 MiB)
837799

## 15 Lattice paths



Starting in the top left corner of a 2×2 grid, and only being able to move to
the right and down, there are exactly 6 routes to the bottom right corner.

How many such routes are there through a 20×20 grid?



### A:



Let's generalize this for a $n\times m$ grid. We always have to go down $n$
times and right $m$ times, but we can choose the order in which to make these
moves. In other words, we are looking for the number of unique sequences of
exactly $n$ "downs" and $m$ "rights". The sequence must be $n+m$ steps,
and we will choose $n$ of those steps to use our "down" moves. The number of
ways in which we can allocate those "down" moves is just a combination of the
following form.

$$ {n+m \choose n} = {n+m \choose m} $$

Since we have an explicit formula, our calculation can be super speedy.



In [1]:
@btime binomial(20+20,20)

216.753 ns (0 allocations: 0 bytes)
137846528820

## 16 Power digit sum



$2^{15}=32768$ and the sum of its digits is $3+2+7+6+8=26$.

What is the sum of the digits of the number $2^{1000}$?



### A:



Again, Julia has support for integers of arbitrary size, so this problem is
trivial.



In [1]:
@btime sum([parse(Int,d) for d in string(big(2)^1000)])

3.295 μs (9 allocations: 3.13 KiB)
1366

## 17 Number letter counts



If the numbers 1 to 5 are written out in words: one, two, three, four, five,
then there are 3 + 3 + 5 + 4 + 4 = 19 letters used in total.

If all the numbers from 1 to 1000 (one thousand) inclusive were written out in
words, how many letters would be used?

NOTE: Do not count spaces or hyphens. For example, 342 (three hundred and
forty-two) contains 23 letters and 115 (one hundred and fifteen) contains 20
letters. The use of "and" when writing out numbers is in compliance with British
usage.



### A:



I didn't like this problem.



## 18 Maximum path sum I



By starting at the top of the triangle below and moving to adjacent numbers
on the row below, the maximum total from top to bottom is 23.

       3
      7 4
     2 4 6
    8 5 9 3

That is, 3 + 7 + 4 + 9 = 23.

Find the maximum total from top to bottom of the triangle below:

                         75
                        95 64
                      17 47 82
                     18 35 87 10
                   20 04 82 47 65
                  19 01 23 75 03 34
                88 02 77 73 07 63 67
                99 65 04 28 06 16 70 92
             41 41 26 56 83 40 80 70 33
            41 48 72 33 47 32 37 16 94 29
          53 71 44 65 25 43 91 52 97 51 14
         70 11 33 28 77 73 17 78 39 68 17 57
       91 71 52 38 17 14 91 43 58 50 27 29 48
      63 66 04 68 89 53 67 30 73 16 69 87 40 31
    04 62 98 27 23 09 70 98 73 93 38 53 60 04 23

NOTE: As there are only 16384 routes, it is possible to solve this problem by
trying every route. However, Problem 67, is the same challenge with a triangle
containing one-hundred rows; it cannot be solved by brute force, and requires a
clever method! ;o)



### A:



First, we're going to need a way to read a triangular matrix from a file because
problem 67 requires it.



In [1]:
"""
Read a triangular matrix from a file or string. Uses an array of arrays rather
than 2d array because the inner arrays aren't uniform length.
"""
function readtriangle(filename::String,T=Int::Type)
    tri = Array{T,1}[]
    open(filename) do file
       for line in eachline(file)
           push!(tri,map(x->parse(T,x),split(line)))
       end
    end
    return tri
end

tri = readtriangle("018.txt")
@test tri[1][1] == 75
@test tri[2] == [95,64]
@test tri[5] == [20,4,82,47,65]

@btime readtriangle("018.txt");

34.635 μs (379 allocations: 16.08 KiB)

To find the maximum possible sum, we could iterate over every possible path. For
a triangle of depth $n$, each path is $n$ nodes long and there are $2^n$
possible paths. Iterating over such a large number isn't ideal, so let's
simplify it. The maximum possible path of the tree with root node, $r$, must
start towards the subtree whose maximum possible path is larger. We can start at
the bottom, taking the maximum of two leaf nodes and adding it to the parents,
and iterating up the tree until we reach the root. This is illustrated on the
four-deep triangular matrix given in the problem statement.

       3           3         3      23
      7 4   ->   7  4   -> 20 19 -> 
     2 4 6     10 13 15
    8 5 9 3

The vast majority of runtime is spent on reading the file.



In [1]:
"""
Find the maximum sum path in a binary tree. Requires tree in a matrix form.
Uses an array of arrays rather than 2d array because the inner arrays aren't
uniform length.
"""
function max_sum_triangle(tri::Array{Array{T,1},1}) where {T<:Number}
    for i in length(tri):-1:2
        for j in 1:(i-1)
            tri[i-1][j] += max(tri[i][j],tri[i][j+1])
        end
    end
    return tri[1][1]
end

tri_test = [[3],[7,4],[2,4,6],[8,5,9,3]]
@test max_sum_triangle(tri_test) == 23

@btime max_sum_triangle(readtriangle("018.txt"))

34.871 μs (379 allocations: 16.08 KiB)
1074

## 19 Counting Sundays



You are given the following information, but you may prefer to do some
research for yourself.

-   1 Jan 1900 was a Monday.
-   Thirty days has September,
    April, June and November.
    All the rest have thirty-one,
    Saving February alone,
    Which has twenty-eight, rain or shine.
    And on leap years, twenty-nine.
-   A leap year occurs on any year evenly divisible by 4, but not on a century
    unless it is divisible by 400.

How many Sundays fell on the first of the month during the twentieth century
(1 Jan 1901 to 31 Dec 2000)?



### A:



[Zeller's congruence](https://en.wikipedia.org/wiki/Zeller's_congruence#Implementation_in_software) will give us the day of the week, $H$ from the day, month,
and year ($D,M,Y$):

$$ H = \left(D + \lfloor 13(M+1)/5 \rfloor + Y + \lfloor Y/4 \rfloor - \lfloor Y/100 \rfloor + \lfloor Y/400 \rfloor \right) \mod 7 $$

The variables $H,M$ behave according to the following dictionaries:

$$ H = \{\text{Sat}:0,\text{Sun}:1,\ldots,\text{Fri}:6\} $$

$$ M = \{\text{Mar}:3,\text{Apr}:4,\ldots,\text{Feb}:14\} $$



In [1]:
"Zeller's congruence for a date array, ``[year,month,day]``."
function zeller(date::Array{T}) where {T<:Integer}
    y = date[2] < 3 ? date[1] - 1 : date[1]
    h = (date[3] + div(13*(MONTH[date[2]]+1),5) + y + div(y,4) - div(y,100) + div(y,400))%7
    return convert(T, h)
end
MONTH = Dict(i=>(i > 2 ? i : i+12) for i in 1:12)

@test zeller([1900,1,1]) == 2
@test zeller([2000,1,1]) == 0
@test zeller([2000,3,1]) == 4

Test Passed

Increment to the first of the next month.



In [1]:
"""
For a date array, ``[year,month,day]``, increment the date to the next first of
the month.
"""
function nextfirst!(D::Array{T}) where {T<:Integer}
    if D[2] == 12
        D[1] += T(1)
        D[2:3] = T[1,1]
    else
        D[2] += T(1)
        D[3] = T(1)
    end
end

date1 = [1901,1,4]
date2 = [2000,12,31]
nextfirst!(date1)
nextfirst!(date2)
@test date1 == [1901,2,1]
@test date2 == [2001,1,1]

Test Passed

Storing dates as `Array` facilitates easy comparison. Loop over all the firsts
with `nextfirst!` and check every first for being a Sunday using `zeller`. This
is much faster than using `tuples` and Python.



In [1]:
"""
Find the number of Sundays that are firsts of the month between two date arrays,
inclusive.
"""
function pe019(d1::Array{T},d2::Array{T}) where {T<:Integer}
    count = 0
    if d1[3] > 1
        nextfirst!(d1)
    end
    while d1 <= d2
        if zeller(d1) == 1
            count += 1
        end
        nextfirst!(d1)
    end
    return count
end

date1 = [1901,1,1]
date2 = [2000,12,31]
@btime pe019(d1,date2) setup=(d1=copy(date1))

1.422 μs (27 allocations: 482 bytes)
171

## 20 Factorial digit sum



n! means n × (n − 1) × &#x2026; × 3 × 2 × 1

For example, 10! = 10 × 9 × &#x2026; × 3 × 2 × 1 = 3628800,
and the sum of the digits in the number 10! is 3 + 6 + 2 + 8 + 8 + 0 + 0 = 27.

Find the sum of the digits in the number 100!



### A:



Trivial because Julia can handle large integers.



In [1]:
"Find the sum of the digits of ``n!``."
function factsum(n::T) where {T<:Integer}
    fact = factorial(big(n))
    return sum([parse(T,d) for d in string(fact)])
end

@test factsum(1) == 1
@test factsum(2) == 2
@test factsum(3) == 6
@test factsum(4) == 6
@test factsum(10) == 27

@btime factsum(100)

2.026 μs (15 allocations: 1.88 KiB)
648